In [1]:
import paddle
import pandas as pd
import numpy as np
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import f1_score, accuracy_score

In [2]:
data_test=pd.read_excel("/home/aistudio/work/评测数据集/test（最终评测集）/真实评测集/usual_test_labeled.xlsx") #测试集
test_sentence=np.array(data_test["文本"])
test_label=np.array(data_test["情绪标签"])

In [3]:
class TokenItem:
    def __init__(self,sentence,tokenizer):
        self.tokenizer=tokenizer
        self.token=tokenizer(
            [sentence],
            max_length=512,
            padding='max_length'
        )
    def __del__(self):
        pass

In [4]:
def get_test_acc(model_path):
    model=AutoModelForSequenceClassification.from_pretrained(model_path,num_classes=6)
    ans = []
    tokenizer=AutoTokenizer.from_pretrained("bert-base-chinese")
    with paddle.no_grad():
        length=len(test_label)
        for i in range(length):
            tokenItem=TokenItem(test_sentence[i],tokenizer)
            outputs = model(input_ids=paddle.to_tensor(tokenItem.token.input_ids,dtype='int64'),
                            token_type_ids=paddle.to_tensor(tokenItem.token.token_type_ids,dtype='int64'))
            del tokenItem #删除对象，节省内存空间
            result = paddle.argmax(outputs[0])
            del outputs #删除对象，节省内存空间
            ans.append(result)
            print("\r{}/{}".format(i+1,length),end='',flush=True)
    return ans

In [5]:
def compute_test(model_path):
    ans=get_test_acc(model_path)
    acc = accuracy_score(test_label, ans)
    f1 = f1_score(test_label, ans, average="macro")
    return {
        "accuracy": acc,
        "f1": f1,
    }

In [6]:
print(compute_test("./work/model/"))

[2024-03-04 15:45:30,253] [    INFO] - We are using <class 'paddlenlp.transformers.bert.modeling.BertForSequenceClassification'> to load './work/model/'.
W0304 15:45:30.269096 14132 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W0304 15:45:30.273743 14132 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[2024-03-04 15:45:33,662] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'bert-base-chinese'.
[2024-03-04 15:45:33,670] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt
[2024-03-04 15:45:33,690] [    INFO] - tokenizer config file saved in /home/aistudio/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2024-03-04 15:45:33,696] [    INFO] - Special tokens file saved in /home/aistudio/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


5000/5000{'accuracy': 0.7552, 'f1': 0.7255935508365438}
